
In this analysis, I need to:

1. Load the Zillow Housing Values data for each ZIP code, selecting only Philadelphia ZIP codes.
2. Calculate the annual average ZHVI for each ZIP code in Philadelphia and each year.
3. Identify the ZIP code with the maximum value for each year.

And the function I use:
1. `pd.read_csv` to read csv file throungh a link (from local or website)
2. dateframe transformation: `merge`, `select`, `rename`, and `drop`
3. how to `groupby` columns
4. create interactive plots by using `altair``


In [2]:
import pandas as pd
import altair as alt
from matplotlib import pyplot as plt

## Step 1: Load the ZHVI data for each ZIP code

In [3]:
df = pd.read_csv("/Users/paprika/Desktop/Courses/MUSA 550 Python/assignment-1-main/data/Zip_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv")
df.head(5)

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,2000-01-31,...,2021-10-31,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-30,2022-05-31,2022-06-30,2022-07-31
0,61639,0,10025,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,329164.0,...,1153364.0,1152736.0,1153314.0,1159292.0,1171216.0,1190200.0,1207107.0,1221417.0,1227148.0,1234232.0
1,84654,1,60657,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,311718.0,...,523727.0,526511.0,528499.0,529879.0,530092.0,532758.0,534840.0,539859.0,543658.0,546709.0
2,61637,2,10023,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,510209.0,...,1517150.0,1521442.0,1521759.0,1532449.0,1542269.0,1559390.0,1572653.0,1591368.0,1600569.0,1607770.0
3,91982,3,77494,Zip,TX,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,224385.0,...,428419.0,435498.0,441231.0,447322.0,456848.0,468519.0,482272.0,493709.0,501230.0,503505.0
4,84616,4,60614,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,407594.0,...,669648.0,674293.0,677215.0,679126.0,678889.0,681982.0,684421.0,690214.0,695114.0,698644.0


## Step 2: Select only Philadelphia ZIP codes

In [4]:
philly = df[(df.City=='Philadelphia') & (df.State=='PA')]
philly = philly.rename(columns={"RegionName": "ZipCode"})
philly = philly.drop(columns={'RegionID','SizeRank','RegionType','StateName','State','City','Metro','CountyName'})
philly.head(5)

,ZipCode,2000-01-31,2000-02-29,2000-03-31,2000-04-30,2000-05-31,2000-06-30,2000-07-31,2000-08-31,2000-09-30,...,2021-10-31,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-30,2022-05-31,2022-06-30,2022-07-31
125,19143,60701.0,61009.0,61132.0,61497.0,62089.0,62804.0,63377.0,63925.0,64313.0,...,173114.0,172087.0,171445.0,171542.0,171680.0,171878.0,171607.0,171333.0,171771.0,172611.0
247,19111,85062.0,85325.0,85511.0,85937.0,86553.0,87106.0,87591.0,87826.0,88161.0,...,257911.0,260104.0,262257.0,263715.0,264809.0,265684.0,267222.0,269460.0,272201.0,274446.0
338,19124,47155.0,47278.0,47408.0,47613.0,47956.0,48226.0,48586.0,48728.0,48955.0,...,156225.0,157780.0,159029.0,159274.0,159886.0,160780.0,161929.0,163625.0,165020.0,166009.0
423,19120,59285.0,59426.0,59583.0,59842.0,60283.0,60535.0,60926.0,61006.0,61287.0,...,161167.0,161807.0,162634.0,162972.0,163597.0,164008.0,164887.0,165860.0,167321.0,168524.0
509,19104,74255.0,74311.0,73420.0,74074.0,75734.0,76763.0,75432.0,72962.0,72727.0,...,220270.0,221454.0,222006.0,220760.0,217933.0,216447.0,216424.0,218663.0,220453.0,223443.0


## Step 3: Calculate the annual average ZHVI for each ZIP code in Philadelphia and each year

In [5]:
ZHVI = pd.melt(philly, id_vars=["ZipCode"],value_vars=philly.columns[1:],value_name="ZHVI", var_name="Years")
ZHVI["Years"] = pd.to_datetime(ZHVI["Years"])
Years = ZHVI["Years"].dt.strftime("%Y")
ZHVI["Years"] = Years

In [6]:
Annual_ZHVI = ZHVI.groupby(['ZipCode', 'Years'])['ZHVI'].mean()
Annual_ZHVI = Annual_ZHVI.reset_index()

In [7]:
bars =( alt.Chart(Annual_ZHVI)
    .mark_bar()
    .encode(
        x=alt.X('Years:Q', scale=alt.Scale(zero=False)),
        y=alt.X('ZHVI:Q', scale=alt.Scale(zero=True)),
        color='ZipCode:N',
        tooltip=['ZipCode', alt.Tooltip('ZHVI', title='ZHVI')]
    ).properties(title='The Annual Average ZHVI For Each ZIP Code in Philadelphia and Each Year',height=500).interactive()
)
bars

alt.Chart(...)

## Step 4: Identify the ZIP code with the maximum value for each year

In [8]:
Annual_MAX = Annual_ZHVI.loc[Annual_ZHVI.groupby(["Years"])["ZHVI"].idxmax()]
Annual_MAX

,ZipCode,Years,ZHVI
207,19118,2000,260641.000000
208,19118,2001,290717.416667
209,19118,2002,323259.250000
210,19118,2003,371517.916667
211,19118,2004,423675.916667
212,19118,2005,486571.500000
213,19118,2006,531186.250000
214,19118,2007,547926.500000
215,19118,2008,533627.250000
216,19118,2009,518370.583333


In [9]:
chart = (
    alt.Chart(Annual_MAX)
    .mark_line(point=True)
    .encode(
        x=alt.X("Years:T",scale=alt.Scale(zero=True)),
        y=alt.Y("ZHVI:Q",scale=alt.Scale(zero=True)),
        tooltip=['ZipCode:Q'],
        color=alt.value("#FAAA66")
    ).properties(title='The ZIP Code With the Maximum Value For Each Year',height=400, width=600)
    .interactive()
)
chart

alt.Chart(...)